### 1. Re-code the house price machine learning

In [1]:
%matplotlib inline

from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import random

data = load_boston()
df = pd.DataFrame(data['data'], columns=data['feature_names'])
y = data['target']
print(df.describe())
print(data['DESCR'])
X = df.TAX # I choose TAX column

             CRIM          ZN       INDUS        CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677083   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         RAD         TAX     PTRATIO           B  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534  356.674032   
std     28.148861    2.1057

###### 1. Random Choose Method to get optimal *k* and *b*

For example, you can change the loss function: $Loss = \frac{1}{n} sum({y_i - \hat{y_i}})^2$ to $Loss = \frac{1}{n} sum(|{y_i - \hat{y_i}}|)$

And you can change the learning rate and observe the performance.

In [2]:
def RMSE(y, y_hat):
    return np.sqrt(sum((y - y_hat) ** 2) / len(y))

def MAE(y, y_hat):
    return sum(abs(y - y_hat)) / len(y)

def loss_random(X, y, n, loss=RMSE):
    loss_min = float('inf')
    k_best, b_best = 0, 0
    for i in range(n):
        k = random.random() * 200 - 100
        b = random.random() * 200 - 100
        y_hat = k * X + b
        loss_new = loss(y, y_hat)
        if loss_new < loss_min:
            loss_min = loss_new
            k_best, b_best = k, b
            print(f"round: {i}, k: {k_best}, b: {b_best}, {loss}: {loss_min}")
        
    return (k_best, b_best)
loss_random(X, y, 2000, RMSE)

round: 0, k: 90.22791033110983, b: -33.979727270317085, <function RMSE at 0x1a206ba488>: 39793.606322708976
round: 1, k: -78.23099737983603, b: 33.680022518843344, <function RMSE at 0x1a206ba488>: 34534.476461913655
round: 2, k: 66.5783416040382, b: 91.39725426536563, <function RMSE at 0x1a206ba488>: 29465.980165567245
round: 4, k: -6.95208816402338, b: -21.306046600959917, <function RMSE at 0x1a206ba488>: 3108.9731826639427
round: 24, k: 1.5141195864932797, b: 60.38866940785607, <function RMSE at 0x1a206ba488>: 705.3895237440589
round: 56, k: 0.714096118313563, b: -51.33926980982337, <function RMSE at 0x1a206ba488>: 250.89116031069332
round: 58, k: 0.4475744719482151, b: -34.95960243452609, <function RMSE at 0x1a206ba488>: 148.6378172855983
round: 95, k: 0.12988800229763342, b: -25.452465891834237, <function RMSE at 0x1a206ba488>: 27.863543555296836


(0.12988800229763342, -25.452465891834237)

###### 2.Supervised Direction to get optimal *k* and *b*

In [3]:
def loss_spvs_dr(X, y, n, alpha=0.1, loss=RMSE):
    loss_min = float('inf')
    direction = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    k = random.random() * 200 - 100
    b = random.random() * 200 - 100
    
    for i in range(n):
        dr_k, dr_b = random.choice(direction) # random walk. Can we walk 4 directions and compare which one is the best?
        k_new = k + dr_k * alpha
        b_new = b + dr_b * alpha
        y_hat = k_new * X + b_new
        loss_new = loss(y, y_hat)
        if loss_new < loss_min:
            k, b = k_new, b_new
            k_best, b_best = k_new, b_new
            loss_min = loss_new
            print(f"round: {i}, k: {k_best}, b: {b_best}, {loss}: {loss_min}")
    return (k_best, b_best)
loss_spvs_dr(X, y, 2000)

round: 0, k: 1.6861944004845468, b: -74.10079430498195, <function RMSE at 0x1a206ba488>: 658.2398655992919
round: 2, k: 1.5861944004845467, b: -74.20079430498194, <function RMSE at 0x1a206ba488>: 614.085385024229
round: 3, k: 1.4861944004845467, b: -74.10079430498195, <function RMSE at 0x1a206ba488>: 570.1249577452724
round: 5, k: 1.3861944004845466, b: -74.20079430498194, <function RMSE at 0x1a206ba488>: 526.0039774293865
round: 7, k: 1.2861944004845465, b: -74.30079430498193, <function RMSE at 0x1a206ba488>: 481.9070185603818
round: 8, k: 1.1861944004845464, b: -74.20079430498194, <function RMSE at 0x1a206ba488>: 438.0182693141807
round: 9, k: 1.0861944004845463, b: -74.30079430498193, <function RMSE at 0x1a206ba488>: 393.9933653128135
round: 15, k: 0.9861944004845463, b: -74.20079430498194, <function RMSE at 0x1a206ba488>: 350.20039983518376
round: 18, k: 0.8861944004845463, b: -74.10079430498195, <function RMSE at 0x1a206ba488>: 306.48602357258034
round: 20, k: 0.7861944004845464, 

(0.18619440048454647, -74.00079430498195)

#### Walk through all 4 directions and find the smallest loss

import numpy as np

def loss_supervise(X, y, n, alpha, loss=RMSE):
    loss_min = float('inf')
    direction = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    k = random.random() * 200 - 100
    b = random.random() * 200 - 100
    
    for i in range(n):
        loss_complete = []
        k_b = []
        for j in range(4):
            k_b.append((k+direction[j][0]*alpha, b+direction[j][1]*alpha))
            loss_complete.append(loss(y, (X*k_b[j][0]+k_b[j][1])))
        k_new, b_new = k_b[loss_complete.index(min(loss_complete))]
        loss_new = min(loss_complete)
        if loss_new < loss_min:
            k, b = minimum[0], minimum[1]
            k_best, b_best = k_new, b_new
            print(f"round: {i}, k: {k_best}, b: {b_best}, {loss}: {loss_min}")
    return (k_best, b_best)
loss_supervise(X, y, 2000, 0.01)

###### 3.Gradient Descent to get optimal *k* and *b*

#### Equations:

$$ RMSE = \frac{1}{n}\sum{(y - (kx+b))^2} = \frac{1}{n}\sum(y^2 -2y(kx+b) + (kx+b)^2)) = \frac{1}{n}\sum{y^2 - 2yxk - 2yb + k^2x^2 + 2kxb + b^2}$$

$$ \frac{\partial{_{loss}}}{\partial{_k}} = \frac{2}{n}(-y + kx + b)x = \frac{2}{n}(-y + \hat{y})x$$

$$ \frac{\partial{_{loss}}}{\partial{_b}} = \frac{2}{n}(-y + kx + b) = \frac{2}{n}(-y + \hat{y})$$

In [11]:
def partial_k(x, y, y_hat):
    n = len(y)
    gradient = 0
    for x_i, y_i, y_hat_i in zip(list(x), list(y), list(y_hat)):
        gradient += (y_i - y_hat_i) * x_i
    return -2 / n * gradient

def partial_b(y, y_hat):
    n = len(y)
    gradient = 0
    for y_i, y_hat_i in zip(list(y), list(y_hat)):
        gradient += (y_i - y_hat_i)
    return -2 / n * gradient

def gradient(X, y, n, alpha=0.01, loss=RMSE):
    loss_min = float('inf')
    
    k = random.random() * 200 - 100
    b = random.random() * 200 - 100
    
    for i in range(n):
        
        y_hat = k * X + b
        loss_new = loss(y, y_hat)
        if loss_new < loss_min:
            loss_min = loss_new
            print(f"round: {i}, k: {k}, b: {b}, {loss}: {loss_min}")
        k_gradient = partial_k(X, y, y_hat)
        b_gradient = partial_b(y, y_hat)
        k += -k_gradient * alpha
        b += -b_gradient * alpha
    return (k, b)
gradient(X, y, 20000)

round: 0, k: -4.48005814490584, b: 75.56712919380263, <function RMSE at 0x1a206ba488>: 1927.787551395356


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


(nan, nan)

###### 4. Try different Loss function and learning rate. 